In [1]:
import pandas as pd   
import numpy as np  
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.stats import levene
from scipy.stats import f_oneway
from scipy.stats import shapiro
from scipy.stats import ttest_ind
import numpy as np
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

# Read CSV, Nashville Crime and Unemployment

In [2]:
nash_crime_df=pd.read_csv('../data/Nashville/Metro_Nashville_Police.csv')

In [3]:
nash_crime_df.head(2)

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Weapon Description,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,07/01/2019 05:31:00 PM,07/01/2019 06:17:00 PM,2410 2410,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218)
1,20180290646_11,20180290646,D,DISPATCHED,A,CLEARED BY ARREST,Closed,03/31/2018 12:21:00 PM,03/31/2018 07:41:00 PM,5824 5824,...,NONE,1,False,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.713 36.042)


In [4]:
nash_unemployment_df=pd.read_csv('../data/Nashville/Unemployment_Nashville-Davidson.csv')                   

In [5]:
nash_unemployment_df.head(2)

,Series ID,Year,Period,Label,Value
0,LAUCA474000000000004,2021,M01,2021 Jan,54892
1,LAUCA474000000000004,2021,M02,2021 Feb,52123


In [6]:
nash_unemployment_df[['Year', 'Month']] = nash_unemployment_df['Label'].str.split(' ', expand=True)
nash_unemployment_df.head()

,Series ID,Year,Period,Label,Value,Month
0,LAUCA474000000000004,2021,M01,2021 Jan,54892,Jan
1,LAUCA474000000000004,2021,M02,2021 Feb,52123,Feb
2,LAUCA474000000000004,2021,M03,2021 Mar,50828,Mar
3,LAUCA474000000000004,2021,M04,2021 Apr,48648,Apr
4,LAUCA474000000000004,2021,M05,2021 May,45838,May


it is interesting to note that Nashville unemployment data is in numbers and not rate or percentage and contains from 2021 - 2023

In [7]:
nash_unemployment_df.rename(columns={'Value': 'Unemployed Population'}, inplace=True)
nash_unemployment_df.tail(2)

,Series ID,Year,Period,Label,Unemployed Population,Month
34,LAUCA474000000000004,2023,M11,2023 Nov,33534,Nov
35,LAUCA474000000000004,2023,M12,2023 Dec,29693,Dec


# Analysis Average unemployment number per month

In [8]:
avg_unemployment_month=round(nash_unemployment_df.groupby('Month')['Unemployed Population'].mean(), 2)
avg_unemployment_month

Month
Apr    34904.67
Aug    36713.67
Dec    29191.00
Feb    39792.33
Jan    40784.67
Jul    39103.00
Jun    43610.00
Mar    37635.00
May    36722.67
Nov    32051.33
Oct    34361.67
Sep    34573.33
Name: Unemployed Population, dtype: float64

In [9]:
month_with_max_unemployment = avg_unemployment_month.idxmax()
month_with_max_unemployment

'Jun'

In [10]:
month_with_min_unemployment = avg_unemployment_month.idxmin()
month_with_min_unemployment

'Dec'

June is the month with the highest unemployment population while December is the lowest

In [11]:
avg_unemployment_year=round(nash_unemployment_df.groupby('Year')['Unemployed Population'].mean(), 2)
avg_unemployment_year

Year
2021    44323.50
2022    32793.50
2023    32743.83
Name: Unemployed Population, dtype: float64

from the analysis, there is an average of more people who are unemployment in 2021 than 2023 and 2023. Remember, the economy was partially shut down in 2021, during the COVID-19 recovery years.

In [12]:
nash_unemployment_df_by_year = nash_unemployment_df.groupby('Year')['Unemployed Population'].sum().reset_index()
nash_unemployment_df_by_year

,Year,Unemployed Population
0,2021,531882
1,2022,393522
2,2023,392926


getting the sum of the unemployment number

In [13]:
nash_unemployment_df_by_year['Year'] = nash_unemployment_df_by_year['Year'].astype('int32')
nash_unemployment_df_by_year.dtypes

Year                     int32
Unemployed Population    int64
dtype: object

In [14]:
nash_crime_df.columns

Index(['Primary Key', 'Incident Number', 'Report Type',
       'Report Type Description', 'Incident Status Code',
       'Incident Status Description', 'Investigation Status',
       'Incident Occurred', 'Incident Reported', 'Incident Location',
       'Latitude', 'Longitude', 'ZIP Code', 'RPA', 'Zone', 'Location Code',
       'Location Description', 'Offense Number', 'Offense NIBRS',
       'Offense Description', 'Weapon Primary', 'Weapon Description',
       'Victim Number', 'Domestic Related', 'Victim Type',
       'Victim Description', 'Victim Gender', 'Victim Race',
       'Victim Ethnicity', 'Victim County Resident', 'Mapped Location'],
      dtype='object')

# Cleaning Zip Code to remove any that does not belong to Nashville

In [36]:
nash_crime_df['ZIP Code'].unique()

array(['37214.', '37013.', '37207.', '37205.', '37076.', '37201.',
       '37208.', '37219.', '37211.', '37138.', '37216.', '37203.',
       '37228.', '37209.', '37210.', '37220.', '37072.', '37218.',
       '37206.', '37115.', '37212.', '37221.', '37217.', '37189.',
       '37213.', '37204.', '37215.', '37240.', '37080.', '37027.',
       '37167.', '37049.', '37025.', '37135.', '37015.', '37122.',
       '37086.', '37143.', '37064.', '37683.', '37232.', '6706.',
       '37130.', '37066.', '3701.', '37129.', '37087.', '37075.',
       '38109.', '30139.', '38562.', '37330.', '37067.', '53701.',
       '27707.', '38401.', '91107.', '27216.', '89502.', '37029.',
       '90069.', '37082.', '0.', '39202.', '37185.', '37043.', '30318.',
       '32801.', '19153.', '43440.', '37110.', '15213.', '37028.',
       '29501.', '23236.', '17202.', '37179.', '37090.', '37078.',
       '37274.', '60123.'], dtype=object)

In [37]:
nash_crime_df.dropna(subset=['ZIP Code'], inplace=True)
nash_crime_df['ZIP Code'].unique()

array(['37214.', '37013.', '37207.', '37205.', '37076.', '37201.',
       '37208.', '37219.', '37211.', '37138.', '37216.', '37203.',
       '37228.', '37209.', '37210.', '37220.', '37072.', '37218.',
       '37206.', '37115.', '37212.', '37221.', '37217.', '37189.',
       '37213.', '37204.', '37215.', '37240.', '37080.', '37027.',
       '37167.', '37049.', '37025.', '37135.', '37015.', '37122.',
       '37086.', '37143.', '37064.', '37683.', '37232.', '6706.',
       '37130.', '37066.', '3701.', '37129.', '37087.', '37075.',
       '38109.', '30139.', '38562.', '37330.', '37067.', '53701.',
       '27707.', '38401.', '91107.', '27216.', '89502.', '37029.',
       '90069.', '37082.', '0.', '39202.', '37185.', '37043.', '30318.',
       '32801.', '19153.', '43440.', '37110.', '15213.', '37028.',
       '29501.', '23236.', '17202.', '37179.', '37090.', '37078.',
       '37274.', '60123.'], dtype=object)

In [38]:
nash_crime_df['ZIP Code'] = nash_crime_df['ZIP Code'].astype(str).str.rstrip('.')
nash_crime_df['ZIP Code'].unique()

array(['37214', '37013', '37207', '37205', '37076', '37201', '37208',
       '37219', '37211', '37138', '37216', '37203', '37228', '37209',
       '37210', '37220', '37072', '37218', '37206', '37115', '37212',
       '37221', '37217', '37189', '37213', '37204', '37215', '37240',
       '37080', '37027', '37167', '37049', '37025', '37135', '37015',
       '37122', '37086', '37143', '37064', '37683', '37232', '6706',
       '37130', '37066', '3701', '37129', '37087', '37075', '38109',
       '30139', '38562', '37330', '37067', '53701', '27707', '38401',
       '91107', '27216', '89502', '37029', '90069', '37082', '0', '39202',
       '37185', '37043', '30318', '32801', '19153', '43440', '37110',
       '15213', '37028', '29501', '23236', '17202', '37179', '37090',
       '37078', '37274', '60123'], dtype=object)

In [39]:
# Assuming 'ZIP Code' is a column in nash_crime_df and you want to remove the last '0' from each ZIP code
nash_crime_df['ZIP Code'] = nash_crime_df['ZIP Code'].astype(str).apply(lambda x: x[:-1] if x.endswith('0') else x)
nash_crime_df['ZIP Code'].unique()

array(['37214', '37013', '37207', '37205', '37076', '37201', '37208',
       '37219', '37211', '37138', '37216', '37203', '37228', '37209',
       '3721', '3722', '37072', '37218', '37206', '37115', '37212',
       '37221', '37217', '37189', '37213', '37204', '37215', '3724',
       '3708', '37027', '37167', '37049', '37025', '37135', '37015',
       '37122', '37086', '37143', '37064', '37683', '37232', '6706',
       '3713', '37066', '3701', '37129', '37087', '37075', '38109',
       '30139', '38562', '3733', '37067', '53701', '27707', '38401',
       '91107', '27216', '89502', '37029', '90069', '37082', '', '39202',
       '37185', '37043', '30318', '32801', '19153', '4344', '3711',
       '15213', '37028', '29501', '23236', '17202', '37179', '3709',
       '37078', '37274', '60123'], dtype=object)

I am applying a function to each element that checks if the ZIP code ends with '0', and then remove it. I tried striping the dots and the zeros came up, which makes it difficult. 

In [40]:
# nash_crime_df = nash_crime_df.dropna(subset=['ZIP Code'])
# nash_crime_df.head(1)

In [41]:
# nash_crime_df['ZIP Code'].unique()

In [42]:
zip_codes_to_remove = ['38109', '3722','27707', '37029', '37683', '37067', '38562', '37025', '37049', '37274', '3724', '3733', '3708', '3713', '3709', '37078','17202','3711','37082','37028','90069','39202','3701','15213','91107','60123','89502','89502', '30139','53701','37043', '19153','27216', '4344','6706','30318', '32801','29501', '23236']
nash_crime_df_filtered = nash_crime_df[~nash_crime_df['ZIP Code'].isin(zip_codes_to_remove)]
nash_crime_df_filtered['ZIP Code'].unique()

array(['37214', '37013', '37207', '37205', '37076', '37201', '37208',
       '37219', '37211', '37138', '37216', '37203', '37228', '37209',
       '3721', '37072', '37218', '37206', '37115', '37212', '37221',
       '37217', '37189', '37213', '37204', '37215', '37027', '37167',
       '37135', '37015', '37122', '37086', '37143', '37064', '37232',
       '37066', '37129', '37087', '37075', '38401', '', '37185', '37179'],
      dtype=object)

In [43]:
nash_crime_df_filtered.dropna(subset=['ZIP Code'])

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Weapon Description,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,07/01/2019 05:31:00 PM,07/01/2019 06:17:00 PM,2410 2410,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218)
1,20180290646_11,20180290646,D,DISPATCHED,A,CLEARED BY ARREST,Closed,03/31/2018 12:21:00 PM,03/31/2018 07:41:00 PM,5824 5824,...,NONE,1,False,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.713 36.042)
2,20200243837_11,20200243837,D,DISPATCHED,U,UNFOUNDED,Closed,04/06/2020 11:15:00 AM,04/06/2020 01:11:00 PM,3144 3144,...,NONE,1,False,G,GOVERNMENT,NaN,NaN,NaN,NaN,POINT (-86.759 36.232)
3,20170471747_11,20170471747,D,DISPATCHED,U,UNFOUNDED,Closed,05/29/2017 02:33:00 AM,05/29/2017 04:27:00 AM,1714 1714,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.907 36.086)
4,20171099391_21,20171099391,D,DISPATCHED,A,CLEARED BY ARREST,Closed,12/16/2017 06:30:00 PM,12/16/2017 10:44:00 PM,1101 1101,...,PERSONAL (HANDS),1,True,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.586 36.178)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865161,20230778865_11,20230778865,T,NaN,U,UNFOUNDED,Closed,12/27/2023 04:00:00 PM,12/30/2023 12:06:00 PM,1708 1708,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.799 36.118)
865163,20230774687_12,20230774687,S,SUSPECT,U,UNFOUNDED,Closed,12/28/2023 10:37:00 AM,12/28/2023 10:37:00 AM,1201 1201,...,NONE,2,False,I,INDIVIDUAL (18 AND OVER),U,U,Unknown,NON RESIDENT,POINT (-86.733 36.158)
865167,20230453680_11,20230453680,D,DISPATCHED,A,CLEARED BY ARREST,Closed,08/03/2023 01:47:00 AM,08/03/2023 09:15:00 AM,426 426,...,HANDGUN - SEMIAUTOMATIC,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Hispanic,NON RESIDENT,POINT (-86.646 36.16)
865169,20230777152_11,20230777152,D,DISPATCHED,U,UNFOUNDED,Closed,12/29/2023 02:52:00 PM,12/29/2023 02:52:00 PM,301 301,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,POINT (-86.815 36.147)


In [44]:
nash_crime_df_filtered['ZIP Code'].unique()

array(['37214', '37013', '37207', '37205', '37076', '37201', '37208',
       '37219', '37211', '37138', '37216', '37203', '37228', '37209',
       '3721', '37072', '37218', '37206', '37115', '37212', '37221',
       '37217', '37189', '37213', '37204', '37215', '37027', '37167',
       '37135', '37015', '37122', '37086', '37143', '37064', '37232',
       '37066', '37129', '37087', '37075', '38401', '', '37185', '37179'],
      dtype=object)

In [48]:
nash_crime_df_filtered.loc[:, 'ZIP Code'] = nash_crime_df_filtered['ZIP Code'].replace('3721', '37210')
nash_crime_df_filtered['ZIP Code'].unique()

array(['37214', '37013', '37207', '37205', '37076', '37201', '37208',
       '37219', '37211', '37138', '37216', '37203', '37228', '37209',
       '37210', '37072', '37218', '37206', '37115', '37212', '37221',
       '37217', '37189', '37213', '37204', '37215', '37027', '37167',
       '37135', '37015', '37122', '37086', '37143', '37064', '37232',
       '37066', '37129', '37087', '37075', '38401', '', '37185', '37179'],
      dtype=object)

In [49]:
nash_crime_df_filtered['ZIP Code'].value_counts()

ZIP Code
37207    46979
37013    43267
37211    42507
37115    37916
37203    27536
37210    26131
37208    25850
37206    23151
37217    22838
37209    22506
37214    20437
37076    20269
37201    12684
37218    11942
37216     9908
37221     9794
37212     6525
37138     6150
37204     5825
37205     5135
37219     5109
37215     4169
37072     3177
37228     2840
37213     2806
37189     2611
37027     2141
37135      238
37015      163
37122       62
37086       26
37143       17
37232        6
37075        5
37087        4
37167        4
37064        3
37129        2
37066        2
38401        1
             1
37185        1
37179        1
Name: count, dtype: int64

In [50]:
nash_crime_df_filtered['Incident Occurred']

0         07/01/2019 05:31:00 PM
1         03/31/2018 12:21:00 PM
2         04/06/2020 11:15:00 AM
3         05/29/2017 02:33:00 AM
4         12/16/2017 06:30:00 PM
                   ...          
865161    12/27/2023 04:00:00 PM
865163    12/28/2023 10:37:00 AM
865167    08/03/2023 01:47:00 AM
865169    12/29/2023 02:52:00 PM
865174    12/28/2023 10:37:00 AM
Name: Incident Occurred, Length: 450739, dtype: object

In [51]:
#convert to datetime
nash_crime_df_filtered.loc[:,'Incident Occurred'] = pd.to_datetime(nash_crime_df_filtered['Incident Occurred'])
nash_crime_df_filtered['Incident Occurred']

/var/folders/th/jsjzrfgj5mqb0ntmxxmc9pmm0000gn/T/ipykernel_69865/2550027996.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nash_crime_df_filtered.loc[:,'Incident Occurred'] = pd.to_datetime(nash_crime_df_filtered['Incident Occurred'])


0         2019-07-01 17:31:00
1         2018-03-31 12:21:00
2         2020-04-06 11:15:00
3         2017-05-29 02:33:00
4         2017-12-16 18:30:00
                 ...         
865161    2023-12-27 16:00:00
865163    2023-12-28 10:37:00
865167    2023-08-03 01:47:00
865169    2023-12-29 14:52:00
865174    2023-12-28 10:37:00
Name: Incident Occurred, Length: 450739, dtype: object

In [55]:
# Convert 'Incident Occurred' to datetime and creating a new column year
nash_crime_df_filtered['Incident Occurred'] = pd.to_datetime(nash_crime_df_filtered['Incident Occurred'], errors='coerce')
nash_crime_df_filtered.loc[:, 'Year'] = nash_crime_df_filtered['Incident Occurred'].dt.year
nash_crime_df_filtered.head(1)


/var/folders/th/jsjzrfgj5mqb0ntmxxmc9pmm0000gn/T/ipykernel_69865/4163609263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nash_crime_df_filtered['Incident Occurred'] = pd.to_datetime(nash_crime_df_filtered['Incident Occurred'], errors='coerce')


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location,Year
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,2019-07-01 17:31:00,07/01/2019 06:17:00 PM,2410 2410,...,1,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218),2019


In [56]:
#extracting and creating a month column
nash_crime_df_filtered.loc[:,'Month'] = nash_crime_df_filtered['Incident Occurred'].dt.month
nash_crime_df_filtered.head(1)

/var/folders/th/jsjzrfgj5mqb0ntmxxmc9pmm0000gn/T/ipykernel_69865/3042958999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nash_crime_df_filtered.loc[:,'Month'] = nash_crime_df_filtered['Incident Occurred'].dt.month


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location,Year,Month
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,2019-07-01 17:31:00,07/01/2019 06:17:00 PM,2410 2410,...,True,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218),2019,7


In [57]:
# nash_crime_df_filtered['Time'] = nash_crime_df_filtered['Incident Occurred'].dt.time
# nash_crime_df_filtered.head(1)

nash_crime_df_filtered.loc[:, 'Time'] = nash_crime_df_filtered['Incident Occurred'].dt.time
nash_crime_df_filtered.head(1)

/var/folders/th/jsjzrfgj5mqb0ntmxxmc9pmm0000gn/T/ipykernel_69865/2390565614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nash_crime_df_filtered.loc[:, 'Time'] = nash_crime_df_filtered['Incident Occurred'].dt.time


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location,Year,Month,Time
0,20190496213_11,20190496213,D,DISPATCHED,U,UNFOUNDED,Closed,2019-07-01 17:31:00,07/01/2019 06:17:00 PM,2410 2410,...,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.696 36.218),2019,7,17:31:00


You would notice i use the .loc which does not give me any warning issues, which i used after i googled it up

In [106]:
crime_count_by_year=nash_crime_df_filtered.groupby('Year').size().reset_index(name='Crime Count')
crime_count_by_year

,Year,Crime Count
0,2017,71019
1,2018,69170
2,2019,61968
3,2020,58112
4,2021,58873
5,2022,60605
6,2023,64029
7,2024,6963


There are more crimes in 2017 that any other year, remember, we are still in 2024. 

In [107]:
year_with_max_crime_count = crime_count_by_year.loc[crime_count_by_year['Crime Count'].idxmax(), 'Year']
year_with_max_crime_count

2017

In [108]:
crime_count_by_year = nash_crime_df_filtered.groupby('Month').size().reset_index(name='Crime Count')
avg_crime_count_by_year = crime_count_by_year['Crime Count'].mean().round(2)
avg_crime_count_by_year

37561.58

# drilling down to 2023

In [118]:

twenty_twenty_three = nash_crime_df_filtered.loc[nash_crime_df_filtered['Year'].isin([2023])]
twenty_twenty_three.head()

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location,Year,Month,Time
144,20230321366_11,20230321366,D,DISPATCHED,U,UNFOUNDED,Closed,2023-06-01 18:30:00,06/01/2023 07:40:00 PM,409 409,...,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,NaN,2023,June,18:30:00
966,20230770706_11,20230770706,D,DISPATCHED,U,UNFOUNDED,Closed,2023-12-25 15:20:00,12/28/2023 03:29:00 PM,41 41,...,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.835 36.091),2023,December,15:20:00
1120,20230761089_21,20230761089,S,SUSPECT,A,CLEARED BY ARREST,Closed,2023-12-20 17:00:00,12/20/2023 06:39:00 PM,405 405,...,S,SOCIETY,NaN,NaN,NaN,NaN,POINT (-86.799 36.158),2023,December,17:00:00
1435,20230774490_11,20230774490,D,DISPATCHED,U,UNFOUNDED,Closed,2023-12-15 09:55:00,12/28/2023 09:55:00 AM,121 121,...,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.72 36.251),2023,December,09:55:00
1449,20230774845_11,20230774845,D,DISPATCHED,U,UNFOUNDED,Closed,2023-12-28 13:15:00,12/28/2023 02:59:00 PM,5824 5824,...,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,POINT (-86.713 36.039),2023,December,13:15:00


In [122]:
twenty_twenty_three_df = twenty_twenty_three.reset_index(drop=True)
twenty_twenty_three_df.head() #64029 rows × 35 columns

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location,Year,Month,Time
0,20230321366_11,20230321366,D,DISPATCHED,U,UNFOUNDED,Closed,2023-06-01 18:30:00,06/01/2023 07:40:00 PM,409 409,...,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,NaN,2023,June,18:30:00
1,20230770706_11,20230770706,D,DISPATCHED,U,UNFOUNDED,Closed,2023-12-25 15:20:00,12/28/2023 03:29:00 PM,41 41,...,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.835 36.091),2023,December,15:20:00
2,20230761089_21,20230761089,S,SUSPECT,A,CLEARED BY ARREST,Closed,2023-12-20 17:00:00,12/20/2023 06:39:00 PM,405 405,...,S,SOCIETY,NaN,NaN,NaN,NaN,POINT (-86.799 36.158),2023,December,17:00:00
3,20230774490_11,20230774490,D,DISPATCHED,U,UNFOUNDED,Closed,2023-12-15 09:55:00,12/28/2023 09:55:00 AM,121 121,...,B,BUSINESS,NaN,NaN,NaN,NaN,POINT (-86.72 36.251),2023,December,09:55:00
4,20230774845_11,20230774845,D,DISPATCHED,U,UNFOUNDED,Closed,2023-12-28 13:15:00,12/28/2023 02:59:00 PM,5824 5824,...,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,POINT (-86.713 36.039),2023,December,13:15:00


converted 2023 sorted crime to df

In [123]:
crime_count_by_month_in_23=twenty_twenty_three_df.groupby('Month').size().reset_index(name='Crime Count')
crime_count_by_month_in_23

,Month,Crime Count
0,April,5356
1,August,5635
2,December,4879
3,February,4840
4,January,5189
5,July,5648
6,June,5599
7,March,5412
8,May,5691
9,November,4852


In [114]:
month_with_max_crime_count_in_23 = crime_count_by_month_in_23.loc[crime_count_by_month_in_23['Crime Count'].idxmax(), 'Month']
month_with_max_crime_count_in_23

'May'

In [124]:
crime_count_by_month = twenty_twenty_three_df.groupby('Month').size().reset_index(name='Crime Count')
avg_crime_count_by_month = crime_count_by_month_in_23['Crime Count'].mean().round(2)
avg_crime_count_by_month

5335.75

# Back to the whole data that includes the whole year

In [111]:
crime_count_by_year = nash_crime_df_filtered.groupby('Month').size().reset_index(name='Crime Count')
avg_crime_count_by_year = crime_count_by_year['Crime Count'].mean().round(2)
avg_crime_count_by_year

37561.58

In [60]:
nash_crime_df_filtered['Month'].dtypes

dtype('int32')

In [61]:
import calendar

nash_crime_df_filtered['Month'] = nash_crime_df_filtered['Month'].apply(lambda x: calendar.month_name[x])


/var/folders/th/jsjzrfgj5mqb0ntmxxmc9pmm0000gn/T/ipykernel_69865/3640405411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nash_crime_df_filtered['Month'] = nash_crime_df_filtered['Month'].apply(lambda x: calendar.month_name[x])


Convert month numbers to month names. i am using the  apply() method to apply a lambda function to each element in the 'Month' column. 

In [74]:
crime_count_by_month=nash_crime_df_filtered.groupby('Month').size().reset_index(name='Crime Count')
crime_count_by_month

,Month,Crime Count
0,April,37356
1,August,38854
2,December,34203
3,February,35916
4,January,41796
5,July,38818
6,June,38040
7,March,37075
8,May,39957
9,November,34260


In [98]:
month_with_max_crime_count = crime_count_by_month.loc[crime_count_by_month['Crime Count'].idxmax(), 'Month']
month_with_max_crime_count

'January'

In [78]:
crime_count_by_month = nash_crime_df_filtered.groupby('Month').size().reset_index(name='Crime Count')
avg_crime_count_by_month = crime_count_by_month['Crime Count'].mean().round(2)

avg_crime_count_by_month

37561.58

In [79]:
crime_count_by_time=nash_crime_df_filtered.groupby('Time').size().reset_index(name='Crime Count')
crime_count_by_time

,Time,Crime Count
0,00:00:00,7853
1,00:01:00,1292
2,00:02:00,182
3,00:03:00,175
4,00:04:00,165
...,...,...
1435,23:55:00,252
1436,23:56:00,149
1437,23:57:00,135
1438,23:58:00,144


In [80]:
crime_count_by_time = nash_crime_df_filtered.groupby('Time').size().reset_index(name='Crime Count')
avg_crime_count_by_time = crime_count_by_time['Crime Count'].mean().round(2)

avg_crime_count_by_time

313.01

In [81]:
# nash_crime_df['ZIP Code'].dtypes

In [ ]:
#df = df.drop(df[df['ZIP Code'].str.startswith('3')].index)
#nash_crime_df.drop(nash_crime_df[nash_crime_df['ZIP Code'].to_string().startswith('3')].index)
#nash_crime_df = nash_crime_df.drop(nash_crime_df[ 30000 < nash_crime_df['ZIP Code'] < 40000].index)

In [ ]:
# list_of_zips = [3701., 6706.0, 53701.0, 27707.0, 91107.0, 27216.0, 89502.0, 90069.0, 0.0, 19153.0, 43440.0, 15213.0, 29501.0, 23236.0, 17202.0, 60123.0]


# for zip in list_of_zips:
#     nash_crime_df.drop(nash_crime_df[nash_crime_df['ZIP Code'] == zip].index, inplace=True)
    

In [ ]:
# nash_crime_df['ZIP Code'].unique()

In [ ]:
# nash_crime_df['ZIP Code'].unique()

In [ ]:
# nash_crime_df_filtered['ZIP Code'].value_counts()

In [ ]:
nash_crime_df_filtered.head()

In [ ]:
nash_crime_df_37207 = nash_crime_df_filtered[nash_crime_df['ZIP Code'] == 37207]
nash_crime_df_37207.size

In [ ]:
crime_count_by_year_37207 = nash_crime_df_37207.groupby('Year').size()
# crime_count_by_year
crime_count_by_year_37207

In [ ]:
nash_crime_df[nash_crime_df['ZIP Code'] == 37214.0].groupby('Year').size()

In [ ]:
crime_count_by_year.reset_index(name='Crime Count')

In [ ]:
crime_count_by_year.name='Crime Count'
crime_count_by_year = crime_count_by_year.reset_index()
crime_count_by_year

In [ ]:
nash_unemployment_df_by_year['Year'] = nash_unemployment_df_by_year['Year'].astype('int32')
nash_unemployment_df_by_year.dtypes

In [ ]:
crime_unemployment_year = pd.merge(nash_unemployment_df_by_year, crime_count_by_year, on='Year', how='inner')
crime_unemployment_year

In [ ]:
#plot barplot to compare unemployment data with crime #ind - bar_width/2, 
bar_width = 0.35
ind = np.arange(len(crime_unemployment_year['Year']))
plt.figure(figsize=(10, 6))
plt.bar(ind - bar_width/2, crime_unemployment_year['Unemployed Population'], bar_width, label='Unemployed Population', color='blue')
plt.bar(ind + bar_width/2, crime_unemployment_year['Crime Count'], bar_width, label='Crime Count', color='Orange')
plt.xlabel('Year')
plt.ylabel('Count')
plt.title('Unemployed Population and Crime Count by Year')
plt.xticks(ind, crime_unemployment_year['Year'])
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
first_list = ['37210', '37207', '37115', '37076', '37013', '37217', '37218',
       '37203', '37211', '37209', '37219', '37206', '37221', '37216',
       '37208', '37214', '37027', '37212', '37204', '37201', '37042',
       '37205', '37189', '37240', '37228', '37213', '37072', '37138',
       '37215', '37220', '37135', '37080', '38401', '37015', '37067',
       '37087', '53701', '37122', '38109', '37075', '60123', '37090',
       '37028', '37086', '37043', '37179', '17202', '23236', '29501',
       '32801', '30318', '0', '37029', '38562', '30139', '89502', '37167',
       '27216', '37129', '37064', '37185', '39202', '37082', '90069',
       '37130', '91107', '27707', '37330', '37274', '37078', '37232',
       '37066', '37110', '43440', '19153', '6706', '15213', '37143',
       '3701', '37683', '37049', '37025']

new_list = []
for zip in first_list:
    if not str(zip).startswith('3'):
        new_list.append(zip)
print(new_list)

In [ ]:
nash_crime_df.head(2)

In [ ]:
nash_crime_df.columns

In [ ]:
# # sex_crimes = nash_crime_df[nash_crime_df['Incident Status Description'].str.contains('SEX CRIMES|RAPE')]


# # sex_crimes = nash_crime_df[nash_crime_df['Incident Status Description'].str.contains('SEX CRIMES|RAPE')]

# sex_crimes = nash_crime_df[nash_crime_df['Incident Status Description'].str.contains('Sex Crime|Rape', case=False, na=False)]

In [ ]:
nash_crime_df['Incident Status Description'].info()

In [ ]:
nash_crime_df.columns

In [ ]:
nash_crime_df['Offense Description'].unique()